# Create proper AI model with LangChain and FAISS as vector database

## Prepare data


## Create vector database in FAISS

Before indexing, you will need to think about the vectorization model. You can use different options: train your own vectorizer (for example, based on models from SentenceTransformers) or use something off-the-shelf. I will use the solution from OpenAI (in this case, you will also need to add an API access key).

As a vector database, I will use FAISS - currently, it is the fastest solution for searching nearest neighbors. In LangChain, there are also implementations for many other storage systems (ElasticSearch, Redis, Pinecone, and so on).

In [37]:
#  import libraries
import os
import pandas as pd
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter # split text into characters
from langchain_openai import OpenAIEmbeddings # vectorize text
from langchain.vectorstores import FAISS


In [38]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [39]:
# upload frame into loader selecting column for vectorization
DATA = os.getenv('DATA')
# loader = TextLoader('/Users/gala/PycharmProjects/Scalian_Chat/ScalianChatBot/Data/www_scalian-spain_es_.txt')
# documents = loader.load()

class TextLoader:
    def __init__(self, folder_path):
        self.folder_path = folder_path

    def load(self):
        documents = []
        for root, _, files in os.walk(self.folder_path):
            for filename in files:
                if filename.endswith('.txt'):
                    file_path = os.path.join(root, filename)
                    with open(file_path, 'r', encoding='utf-8') as file:
                        documents.append(file.read())
        return documents

folder_path = DATA
loader = TextLoader(folder_path)
documents = loader.load()

In [40]:
documents

['title: Energía | Scalian Spain\nurl: https://www.scalian-spain.es/experiencias-de-negocio/energia/\n\n\n\nEnergía\n\n\nAyudamos a revolucionar el modo en que las empresas energéticas gestionan los datos para ofrecer mejores servicios al usuario y ser más eficientes en la toma de decisiones.\n\n\nAprender de los datos para reinventarse.\n\n\nCompañías de energía conviven en un entorno sumamente competitivo donde la transformación es la clave que les permitirá\xa0 aumentar su rentabilidad y destacar. Ya sean Productoras, Distribuidoras o Comercializadoras»\xa0 todas deben acelerar su proceso de digitalización que les permita una mejor toma de decisiones basadas en sus propios datos. Una realidad que ya ha empezado y no pueden seguir aplazando.\n\n\nOptimización de procesos complejos  para mejorar la eficiencia.\n\n\nLos trabajos en campo, tanto para labores de mantenimiento como para labores de operación o lectura, son uno de los elementos de mayor coste para las compañías energéticas.

In [41]:
# concatenate all documents into one
doc = ''.join(documents)
# clean text
doc = doc.replace('\n', ' ').replace('\xa0', ' ')
doc

'title: Energía | Scalian Spain url: https://www.scalian-spain.es/experiencias-de-negocio/energia/    Energía   Ayudamos a revolucionar el modo en que las empresas energéticas gestionan los datos para ofrecer mejores servicios al usuario y ser más eficientes en la toma de decisiones.   Aprender de los datos para reinventarse.   Compañías de energía conviven en un entorno sumamente competitivo donde la transformación es la clave que les permitirá  aumentar su rentabilidad y destacar. Ya sean Productoras, Distribuidoras o Comercializadoras»  todas deben acelerar su proceso de digitalización que les permita una mejor toma de decisiones basadas en sus propios datos. Una realidad que ya ha empezado y no pueden seguir aplazando.   Optimización de procesos complejos  para mejorar la eficiencia.   Los trabajos en campo, tanto para labores de mantenimiento como para labores de operación o lectura, son uno de los elementos de mayor coste para las compañías energéticas.Scalian dispone de una gran

In [42]:
# split text into characters, to meet the token limit, in our case it's not necessary
splitter = RecursiveCharacterTextSplitter(chunk_size=2000,chunk_overlap=0)
texts = splitter.split_text(doc)

In [43]:
texts[0]

'title: Energía | Scalian Spain url: https://www.scalian-spain.es/experiencias-de-negocio/energia/    Energía   Ayudamos a revolucionar el modo en que las empresas energéticas gestionan los datos para ofrecer mejores servicios al usuario y ser más eficientes en la toma de decisiones.   Aprender de los datos para reinventarse.   Compañías de energía conviven en un entorno sumamente competitivo donde la transformación es la clave que les permitirá  aumentar su rentabilidad y destacar. Ya sean Productoras, Distribuidoras o Comercializadoras»  todas deben acelerar su proceso de digitalización que les permita una mejor toma de decisiones basadas en sus propios datos. Una realidad que ya ha empezado y no pueden seguir aplazando.   Optimización de procesos complejos  para mejorar la eficiencia.   Los trabajos en campo, tanto para labores de mantenimiento como para labores de operación o lectura, son uno de los elementos de mayor coste para las compañías energéticas.Scalian dispone de una gran

In [44]:
# vectorize text
openai_api_key = os.getenv("OPENAI_API_KEY")
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key, model="text-embedding-3-small")

In [45]:
# create vector database
db = FAISS.from_texts(texts, embeddings)
db.as_retriever()
# and save it locally
db.save_local("faiss_db_scilian")


In [46]:
# testing retriever
db.similarity_search_with_score("What strategies does Scalian Spain employ to ensure a smooth transition to new operational management paradigms, and how do they integrate digitalization, performance management, and expertise to achieve this goal?", top_k=3)

[(Document(page_content='propósito de Scalian   Gobernanza   Encuéntrenos   Ver todo   Newsroom   Insights   Buscar sobre Scalian   Home ES - Expertises - Estrategia y Transformación   Estrategia y Transformación   La combinación de nuestras experiencias en gestión y transformacion digital nos permite intervenir, tanto en la definición de esquemas de transformación como en el acompañamiento y el despliegue de soluciones.   Introducción   Newsroom   Enfoque   Competencias   Case studies   Nuestros expertos   Contacto    - ¿Cómo garantizar el éxito de un proceso de transformación?  - Nuestro enfoque pragmático de la gestión digital y el enfoque innovador de la gestión de la transformación garantizan a nuestros clientes una trayectoria de transformación que tenga en cuenta su contexto y las evoluciones tecnológicas, sectoriales y sociales. Por último, siempre tenemos en cuenta el elemento humano en nuestras decisiones organizativas, para garantizar la eficacia de los cambios que realizamo

We have confirmed that our vector search retrieves relevant documents. Now our task is to pass them to the LLM for obtaining a more detailed and human-like response.

## Lets add power of chartGPT to our retriever

In LangChain, this can be done in several ways:
1. Build a chain load_qa_chain from the responses of our retriever.
2. Directly access the vector database RetrievalQA.
3. Obtain specific context and pass it to the LLMChain in the LLM chain.


### Using 1 option  with RetrievalQA

In [47]:
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI


In [48]:
# create chain
chain = RetrievalQA.from_chain_type(llm=OpenAI(openai_api_key=openai_api_key, temperature=0), retriever=db.as_retriever(), chain_type='stuff')

In [49]:
query = "What strategies does Scalian Spain employ to ensure a smooth transition to new operational management paradigms, and how do they integrate digitalization, performance management, and expertise to achieve this goal?"
chain.invoke(query)

{'query': 'What strategies does Scalian Spain employ to ensure a smooth transition to new operational management paradigms, and how do they integrate digitalization, performance management, and expertise to achieve this goal?',
 'result': ' Scalian Spain employs a combination of digitalization, performance management, and expertise to ensure a smooth transition to new operational management paradigms. This includes using their expertise in management and digital transformation to define transformation schemes and accompany clients in deploying solutions. They also prioritize the human element in organizational decisions to ensure the effectiveness of changes. Additionally, they use a pragmatic approach to digital management and an innovative approach to transformation management to consider the context and technological, sectoral, and social evolutions. They also have a methodology equipped to support clients in this change, utilizing technologies such as big data, artificial intellige

In [50]:
query = "Miembros del equipo de gestión de Scalian Group con sus titulos?"
chain.invoke(query)

{'query': 'Miembros del equipo de gestión de Scalian Group con sus titulos?',
 'result': '\n1. Yvan Chabanne - Presidente del Grupo Scalian\n2. Jean-Charles de Borda - Chief Operations Officer Europe\n3. Fanny Mercieca - Mergers & Acquisitions Director\n4. Christophe Reybert - Chief Operations Officer France\n5. David Capdaspe Lagangue - Director Administrativo y Financiero\n6. Caroline Nancy - Directora de Recursos Humanos\n7. Jean-Manuel Silva - Secretario General y Director Jurídico\n8. Marie de Saint Salvy - Senior Vice-President and member of the Executive Committee'}

In [51]:
query1 = "Does 'Scalian' organize workshops and werbinars?"
chain.invoke(query1)

{'query': "Does 'Scalian' organize workshops and werbinars?",
 'result': ' Yes, Scalian offers various opportunities for employees to participate in events, projects, workshops, and webinars that promote the exchange of ideas, practices, and experiences. These events are open to all employees and are designed to inspire, share, and learn.'}

In [52]:
query ="Quelles sont les principales étapes du déploiement d'un projet de données, selon la description donnée dans le texte, et comment le Centre d'Excellence en Big Data Ariane aide-t-il à accélérer ces projets?"
chain.invoke(query)

{'query': "Quelles sont les principales étapes du déploiement d'un projet de données, selon la description donnée dans le texte, et comment le Centre d'Excellence en Big Data Ariane aide-t-il à accélérer ces projets?",
 'result': " Selon la description donnée dans le texte, les principales étapes du déploiement d'un projet de données sont la détermination de la stratégie de données, la mise en place de l'infrastructure et l'intégration des données, la mise en œuvre des processus d'ingénierie de données, la création de cas d'utilisation et la mise en œuvre de la solution. Le Centre d'Excellence en Big Data Ariane aide à accélérer ces projets en offrant une vision unique des flux de données, en améliorant la supervision et la gestion des erreurs, en permettant l'orchestration de tous les processus de données et en aidant à déterminer la stratégie de données et à sélectionner la plateforme Big Data la plus adaptée."}

## 2 option with LLMChain  and Prompt modification if needed

In [53]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [77]:
# create prompt template
prompt_template = """
You first greet the client, then answer the question.
Use the context to answer the question, following these rules:
Answer the question on the language of the question.
If the question on English, answer on English.
If the question on Spanish, answer on Spanish.
If the question on French, answer on French.
Do not display the context.
{context}

Question: {input}
Answer:
url: {url}
"""
# context is the text of the document and the question is the input/query
prompt = PromptTemplate.from_template(template=prompt_template)
prompt.input_variables

['context', 'input', 'url']

In [84]:
# create chain with custom prompt
chain = LLMChain(llm=OpenAI(temperature=0, openai_api_key=openai_api_key, max_tokens=500), prompt=prompt)
# temperature=0 - means that we will get the same answer every time

In [85]:
query2 = "Quelles sont les principales étapes du déploiement d'un projet de données, selon la description donnée dans le texte, et comment le Centre d'Excellence en Big Data Ariane aide-t-il à accélérer ces projets?"

In [86]:
# retrieve answer from database
relevants = db.similarity_search(query2, k=1)

## get answer and url in dict format
doc = relevants[0].page_content

In [87]:
print(doc)

necesidades.   Formación del equipo y transferencia de conocimiento   Sesiones formativas en Big Data para transferir conocimiento a tu departamento técnico para que seáis autosuficientes.   0%   de esfuerzo   Descubre el Centro de Excelencia de Big DataAriane: Nuestro Catalizador de Proyectos de Datos   “Ariane recopila nuestro knowhow, metodología y herramientas que se unen para acelerar los proyectos de Analítica del Big Data”Ariane ha nacido con la idea de ser un marco de trabajo sencillo para la industrialización y automatización de los trabajos de big data e inteligencia artificial que orqueste de forma centralizada cada proceso dentro del ciclo de vida desde el dato al resultado que es capaz de integrar mediante la gestión de la configuración una gran variedad de soluciones para cualquier entorno empresarial.   -   %   Costes Operativos   +   %   Velocidad en la ejecución de proyectos   Beneficios de Ariane   Ofrece una visión única de los flujos de datos en todas las etapas del

In [88]:
# response from LLM in new prompt format
# invoke chain by passing the input as a dictionary
response=chain.invoke({'context': doc, 'input': query2, 'url': doc})



In [89]:
print(response)

{'context': 'necesidades.   Formación del equipo y transferencia de conocimiento   Sesiones formativas en Big Data para transferir conocimiento a tu departamento técnico para que seáis autosuficientes.   0%   de esfuerzo   Descubre el Centro de Excelencia de Big DataAriane: Nuestro Catalizador de Proyectos de Datos   “Ariane recopila nuestro knowhow, metodología y herramientas que se unen para acelerar los proyectos de Analítica del Big Data”Ariane ha nacido con la idea de ser un marco de trabajo sencillo para la industrialización y automatización de los trabajos de big data e inteligencia artificial que orqueste de forma centralizada cada proceso dentro del ciclo de vida desde el dato al resultado que es capaz de integrar mediante la gestión de la configuración una gran variedad de soluciones para cualquier entorno empresarial.   -   %   Costes Operativos   +   %   Velocidad en la ejecución de proyectos   Beneficios de Ariane   Ofrece una visión única de los flujos de datos en todas l